In [1]:
import scrapy
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from scrapy.selector import Selector
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
 

search_str = 'cybersecurity financial industry'
last_x_weeeks = 4

### 1. Search terms defined abvove

In [2]:
allowed_domains = ['www.cnn.com']
start_urls = ['https://www.cnn.com']

# set up the driver
chrome_options = Options()
# chrome_options.add_argument("--headless") # uncomment if don't want to appreciate the sight of a possessed browser
driver = webdriver.Chrome(chrome_options)
driver.get("https://edition.cnn.com/search")



search_input = driver.find_element(by = By.XPATH,value="//*[@id='search']/div[1]/div[1]/input") # find the search bar
search_input.send_keys(search_str) # type in the search term

time.sleep(1)
search_btn = driver.find_element(by = By.XPATH,value = "//*[@id='search']/div[1]/div[1]/button[2]") # find the search button
search_btn.click()

time.sleep(1)
stories_label = driver.find_element(by = By.XPATH,value = "//*[@id='search']/div[1]/div[2]/div/div/ul/li[2]/label") # find the search button
stories_label.click()

time.sleep(1)
search_btn = driver.find_element(by = By.XPATH,value = "//*[@id='relevance']") # find the search button
search_btn.click()

time.sleep(2)
driver.refresh()
time.sleep(2)

html = [driver.page_source]

### 2. Define function to scrap each news page

In [3]:
def get_pages(html):
    rows = []
    for page in html:
        resp = Selector(text = page)
        #print(resp)
        results = resp.css("#search > div.search__right > div > div.search__results-list > div > div.container_list-images-with-description__cards-wrapper > div > div") # result iterator
        #print(results)    
        dataUriList =results.xpath("//*[@id='search']/div[2]/div/div[2]/div/div[2]/div/div")
        news_url = dataUriList.xpath("//*[@class='container__headline-text']")
        #print(news_url)
        row = []
        for i,item in enumerate(news_url):
            
            
            title = item.xpath(".//text()").get()
            #print(title)

            
            date_div = item.xpath(f"//*[@id='search']/div[2]/div/div[2]/div/div[2]/div/div/div[{i+1}]/a[2]/div/div[2]")
            date = date_div.xpath(".//text()").get().strip()
            datetime_parsed =datetime.strptime(date, "%b %d, %Y")
            #print(date)


            link = item.xpath(".//@data-zjs-href").get()
            #print(link)


            description_div = item.xpath(f"//*[@id='search']/div[2]/div/div[2]/div/div[2]/div/div/div[{i+1}]/a[2]/div/div[3]")
            description = description_div.xpath(".//text()").get().strip()
            #print(description)
            author = 'CNN'
            row = [title,datetime_parsed,description,link,author]
            rows.append(row)

        return rows

### 3. Poll each page

In [4]:
headers = ['Title','Publish Date','Description','Url','Author']
news = get_pages(html)

In [5]:
for i in range(3):
    
    i+=1
    time.sleep(2)
    next_btn = driver.find_element(by = By.XPATH,value = "//*[@id='search']/div[2]/div/div[4]/div/div[3]")
    next_btn.click()
    time.sleep(2)
    driver.refresh()
    time.sleep(2)
    news.extend(get_pages([driver.page_source]))

print(len(news))


40


### 4. Order and filter news by date

In [8]:
filter_date = datetime.today()-timedelta(weeks=last_x_weeeks)


news_df = pd.DataFrame(data=news,columns=headers)
news_df_sorted = news_df.sort_values(by= 'Publish Date', ascending=False)
news_df_filtered = news_df_sorted[news_df_sorted['Publish Date']>=filter_date]

### 5. Build the output file to be used as chat gpt input

In [9]:
 
print(news_df_filtered)
news_df_filtered.to_csv('OutputTestFiles/cnn_test.csv',sep=";")

                                                Title Publish Date  \
37  Cyberattack disrupts operations at major US he...   2024-05-08   
17  US government calls for insurance companies to...   2024-03-10   
3   ‘We’re hemorrhaging money’: US health clinics ...   2024-03-09   
8   Top US cybersecurity agency hacked and forced ...   2024-03-08   
26  Cyberattack on insurance giant disrupting busi...   2024-03-01   
1   FTC fines cybersecurity company Avast $16.5 mi...   2024-02-22   

                                          Description  \
37  A cyberattack has disrupted “clinical operatio...   
17  The US government on Sunday urged insurance co...   
3   For more than two weeks, a cyberattack has dis...   
8   A federal agency in charge of cybersecurity di...   
26  A week after a cyberattack disrupted insurance...   
1   Cybersecurity software company Avast faces a $...   

                                                  Url Author  
37  https://www.cnn.com/2024/05/08/tech/cybera